# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおける「20の質問」ゲームのために設計されたAIエージェントを開発するためのコードを含んでいます。特に、質問者（GemmaQuestionerAgent）と回答者（GemmaAnswererAgent）の二つのエージェントを実装しており、エージェントは交互に質問を行い、またはその質問に対する「はい」または「いいえ」の応答を行います。

### 問題の概要
「20の質問」ゲームにおいて、エージェントは、限られた質問を通じてターゲットワードを推測するという課題に取り組んでいます。質問者は、ユーザーからの答えを元にターゲットワードを推測するための質問を出し、回答者は与えられた質問に対して適切な応答を行います。

### アプローチと使用ライブラリ
1. **Gemmaライブラリ**: Googleが開発したGemmaというPyTorchで実装された因果モデルを使用しており、自然言語処理における生成タスクを扱います。Gemmaの設定やトークナイザーの構成が行われています。

2. **PyTorch**: モデルのトレーニングと推論のために利用されており、GPU（CUDA）が使用されています。これにより、大規模な言語モデルの生成タスクを効率的に実行できます。

3. **観察クラス**: ゲームの進行状況を保持するための`obs`クラスが定義されており、ターンタイプ（質問、回答、推測）、キーワード、カテゴリ、質問リスト、応答リストなどの属性を持ちます。これはエージェントが状態を管理し、次のアクションを決定するための重要な役割を果たします。

4. **クラスベースの設計**: `GemmaFormatter`や、質問者エージェント、回答者エージェントの各クラスは、ゲームの進行に応じてプロンプトをフォーマットし、対話の状態を管理します。これにより、エージェント間のインタラクションがスムーズになります。

5. **ハイパーパラメータと出力の調整**: 質問と推測の生成過程では、温度やトークン制限といったハイパーパラメータも設定されており、生成された応答の多様性が制御されます。

このノートブックは、抽象化されたクラス構造を通じて、AIエージェントが「20の質問」ゲームを効果的にプレイできるような仕組みを提供しています。これにより、エージェントがゲームのルールを遵守しつつ、戦略的に質問や推測を行う能力が発揮されることを目指しています。

---


# 用語概説 
このJupyterノートブックの内容に基づいて、機械学習や深層学習の初心者がつまずきそうな専門用語について解説します。また、有名なものや初心者でも比較的理解しやすい概念については説明しません。

1. **Gemma**:
   - Gemmaは、特定のタスクに対するCausal Language Model（因果言語モデル）です。このモデルは、与えられたプロンプトに基づいて次の単語を生成することを目的としています。本ノートブックでは、20の質問ゲームの文脈で使用されます。

2. **Causal LM (因果言語モデル)**:
   - 自然言語処理の一種のモデルで、過去の単語を見ることによって次に来る単語を予測します。さまざまな文のコンテキストに基づく逐次的な生成タスクに使われます。

3. **few-shot examples (少数ショット例)**:
   - モデルが新しいタスクを学ぶ際に、少数の例示をプロンプトとして与える手法です。これは、モデルにとって不明な質問の形式や期待される回答のスタイルを示すために使用されます。

4. **パラメータの量（7b-IT-quant / 2b）**:
   - 「7b」は、モデルが持つパラメータの数（70億）を指し、「2b」は、2億のパラメータを持つモデルを指します。量が多いほど、モデルが学습できるパターンの数が増え、一般的には性能が向上しますが、計算リソースも多く必要とします。

5. **Quantization (量子化)**:
   - モデルのサイズを縮小し、計算の効率を向上させる技術です。浮動小数点数の代わりに整数を使って、モデルの重みや出力を圧縮します。これにより、低メモリデバイスでも動かすことが可能となる場合があります。

6. **contextlib (コンテキストマネージャ)**:
   - Pythonの標準ライブラリで提供される構文で、リソース管理やクリーンアップを簡単に行うための機能です。特定のブロック内でのみ適用される設定や、リソースの管理（例: ファイルを開く、データベースを接続）を行うのに使われます。

7. **torch.set_default_dtype**:
   - PyTorchにおいて、デフォルトのテンソルデータ型を設定するために使用される関数。これにより、モデルの性能やメモリ使用量に影響を与えることがあります。

8. **interleave_unequal**:
   - 異なる長さのリストを交互に結合するカスタム関数です。例えば、質問と回答のリストが異なる長さの場合でも、均等に応じた形で結果をまとめることができます。ミスマッチした長さのリストを処理する際に役立ちます。

9. **obs 클래스**:
   - ゲームの状態や観察結果を管理するためのクラス。エージェントに渡すための状態（ターンの種類、キーワード、カテゴリ、質問、回答）を扱います。この設計は、ゲームの進行状況やエージェントの振る舞いを把握するのに便利です。

10. **記号による回答フォーマット（**）**:
    - 特定の形式（この場合は二重アスタリスク）で答えを囲むことによって、モデルが期待する形式での応答を強制する手法です。このプロンプトを通じて、エージェントに明確な形式を求めます。

これらの用語は、初心者にとっては新しい概念であるか、特定の文脈での使われ方を理解するのが難しい場合がありますので、丁寧に学ぶ必要があります。

---


In [ ]:
"""
あなたの提出コードはここに記述する必要があります。 
魔法のコマンドをコメントアウトして、pyファイルと提出コードを書き込みます。
シミュレートされたゲーム値を渡すことができるobsクラスを作成するコードブロックを最後に追加します。
GPUアクセラレータを有効にして実行します。
以下に参照用のスターターノートブックが追加されています。
"""

## スターターノートブックの例
テスターを早く見るために2bに変更されただけです。 


https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook?kernelSessionId=178755035


In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
#%%writefile submission/main.py
# 設定
import os
import sys

# **重要:** このようにシステムパスを設定して、ノートブック内およびシミュレーション環境でコードが機能するようにします。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """与えられたdtypeにデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を担います。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいまたはいいえで答える質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを予想してください。予想は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を担います。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"質問はキーワード{obs.keyword}とカテゴリ{obs.category}に関するものです。はいまたはいいえで答えてください。回答は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者のはいまたはいいえの質問に答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を担います。まずの質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** では、キーワードを予想してください。",
    "**フランス**", "正解です！",
]


# **重要:** グローバル変数としてエージェントを定義し、必要なエージェントのみロードします。
# 両方をロードすると、メモリ不足 (OOM) になる可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "はい"
    else:
        return response

提出ファイルの作成をコメントアウトしました


In [ ]:
# !apt install pigz pv > /dev/null
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

## クイックテスト

これは質問者と推測者エージェントに適用されます。


In [ ]:
class obs(object):
    '''
    このクラスはゲームからの観察を表します。 
    turnTypeは['ask','answer','guess']のいずれかであることを確認してください。
    テストしたいキーワードを選択してください。
    キーワードのカテゴリを選択してください。
    質問は文字列のリストである必要があります。エージェントに質問の履歴を渡す場合はこれを使用します。
    回答も文字列のリストである必要があります。
    回答者エージェントに質問を渡すために response を使用します。
    '''
    def __init__(self, turnType, keyword, category, questions, answers):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers

# エージェントに渡すobsの例。こちらに自分のobsを渡してください。
obs1 = obs(turnType='ask', 
          keyword='paris', 
          category='place', 
          questions=['それは人ですか？', 'それは場所ですか？', 'それは国ですか？', 'それは都市ですか？'], 
          answers=['いいえ', 'はい', 'いいえ', 'はい'])

question = agent_fn(obs1, None)
print(question)

# 出力としてエージェントからの応答を得る必要があります。 
# これは提出エラーに役立ちませんが、エージェントの応答を評価するのには役立ちます。

In [ ]:
obs2 = obs(turnType='guess', 
          keyword='paris', 
          category='place', 
          questions=['それは人ですか？', 'それは場所ですか？', 'それは国ですか？', 'それは都市ですか？', 'そのキーワードはたくさんの雨がありますか？'], 
          answers=['いいえ', 'はい', 'いいえ', 'はい', 'いいえ'])

agent_fn(obs2, None)